# Miniproject 1: Image Classification

## Introduction

### Description

One of the deepest traditions in learning about deep learning is to first [tackle the exciting problem of MNIST classification](http://deeplearning.net/tutorial/logreg.html). [The MNIST database](https://en.wikipedia.org/wiki/MNIST_database) (Modified National Institute of Standards and Technology database) is a large database of handwritten digits that was [recently extended](https://arxiv.org/abs/1702.05373). We break with this tradition (just a little bit) and tackle first the related problem of classifying cropped, downsampled and grayscaled images of house numbers in the [The Street View House Numbers (SVHN) Dataset](http://ufldl.stanford.edu/housenumbers/).


### Prerequisites

- You should have a running installation of [tensorflow](https://www.tensorflow.org/install/) and [keras](https://keras.io/).
- You should know the concepts "multilayer perceptron", "stochastic gradient descent with minibatches", "training and validation data", "overfitting" and "early stopping".

### What you will learn

- You will learn how to define feedforward neural networks in keras and fit them to data.
- You will be guided through a prototyping procedure for the application of deep learning to a specific domain.
- You will get in contact with concepts discussed later in the lecture, like "regularization", "batch normalization" and "convolutional networks".
- You will gain some experience on the influence of network architecture, optimizer and regularization choices on the goodness of fit.
- You will learn to be more patient :) Some fits may take your computer quite a bit of time; run them over night.

### Evaluation criteria

The evaluation is (mostly) based on the figures you submit and your answer sentences. 
We will only do random tests of your code and not re-run the full notebook.

### Your names

Before you start, please enter your full name(s) in the field below; they are used to load the data. The variable student2 may remain empty, if you work alone.

In [ ]:
student1 = 285467
student2 = 233984

## Some helper functions

For your convenience we provide here some functions to preprocess the data and plot the results later. Simply run the following cells with `Shift-Enter`.

### Dependencies and constants

In [ ]:
import numpy as np
import scipy.io
import matplotlib.pyplot as plt

# Olesya's imports
import json
import time
import timeit
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
from matplotlib.pyplot import cm
fig_size = 12, 8
plt.rcParams["figure.figsize"] = fig_size
plt.style.use('ggplot')
%matplotlib inline

# Leonore's imports
from keras import backend as K
from keras.layers import BatchNormalization, regularizers
# ...

import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten
from keras.optimizers import SGD, Adam


# you may experiment with different subsets, 
# but make sure in the submission 
# it is generated with the correct random seed for all exercises.
np.random.seed(hash(student1 + student2) % 2**32)
subset_of_classes = np.random.choice(range(10), 5, replace = False)

### Plotting

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] = 10, 6
def plot_some_samples(x, y = [], yhat = [], select_from = [], 
                      ncols = 6, nrows = 4, xdim = 16, ydim = 16,
                      label_mapping = range(10)):
    """plot some input vectors as grayscale images (optionally together with their assigned or predicted labels).
    
    x is an NxD - dimensional array, where D is the length of an input vector and N is the number of samples.
    Out of the N samples, ncols x nrows indices are randomly selected from the list select_from (if it is empty, select_from becomes range(N)).
    
    Keyword arguments:
    y             -- corresponding labels to plot in green below each image.
    yhat          -- corresponding predicted labels to plot in red below each image.
    select_from   -- list of indices from which to select the images.
    ncols, nrows  -- number of columns and rows to plot.
    xdim, ydim    -- number of pixels of the images in x- and y-direction.
    label_mapping -- map labels to digits.
    
    """
    fig, ax = plt.subplots(nrows, ncols)
    if len(select_from) == 0:
        select_from = range(x.shape[0])
    indices = np.random.choice(select_from, size = min(ncols * nrows, len(select_from)), replace = False)
    for i, ind in enumerate(indices):
        thisax = ax[i//ncols,i%ncols]
        thisax.matshow(x[ind].reshape(xdim, ydim), cmap='gray')
        thisax.set_axis_off()
        if len(y) != 0:
            j = y[ind] if type(y[ind]) != np.ndarray else y[ind].argmax()
            thisax.text(0, 0, (label_mapping[j]+1)%10, color='green', 
                                                       verticalalignment='top',
                                                       transform=thisax.transAxes)
        if len(yhat) != 0:
            k = yhat[ind] if type(yhat[ind]) != np.ndarray else yhat[ind].argmax()
            thisax.text(1, 0, (label_mapping[k]+1)%10, color='red',
                                             verticalalignment='top',
                                             horizontalalignment='right',
                                             transform=thisax.transAxes)
#    return fig # Mute for %matplotinline

def prepare_standardplot(title, xlabel):
    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.suptitle(title)
    ax1.set_ylabel('Categorical cross-entropy')
    ax1.set_xlabel(xlabel)
    ax1.set_yscale('log')
    ax2.set_ylabel('Accuracy [% correct]')
    ax2.set_xlabel(xlabel)
    return fig, ax1, ax2

def finalize_standardplot(fig, ax1, ax2):
    ax1handles, ax1labels = ax1.get_legend_handles_labels()
    if len(ax1labels) > 0:
        ax1.legend(ax1handles, ax1labels)
    ax2handles, ax2labels = ax2.get_legend_handles_labels()
    if len(ax2labels) > 0:
        ax2.legend(ax2handles, ax2labels)
    fig.tight_layout()
    plt.subplots_adjust(top=0.9)

def plot_history(history, title):
    fig, ax1, ax2 = prepare_standardplot(title, 'epoch')
    ax1.plot(history.history['loss'], label = "training")
    ax1.plot(history.history['val_loss'], label = "validation")
    ax2.plot(history.history['acc'], label = "training")
    ax2.plot(history.history['val_acc'], label = "validation")
    finalize_standardplot(fig, ax1, ax2)
#    return fig # Mute for %matplotinline

### Additional Helpers

In [ ]:
def get_legend(colormap, vars_, lines=True):
    color_iter = iter(colormap)
    color=next(color_iter)
    legend = []
    if lines:
        legend.append(mlines.Line2D([], [], color=color, linestyle='--', label='Training'))
        legend.append(mlines.Line2D([], [], color=color, alpha=0.3, label='Validation'))

    color_iter = iter(colormap)
    for var in vars_:
        legend.append(mpatches.Patch(color=next(color_iter), label=var))

    return legend

def plot(histories, labels, title, mode, complementary=True):
        get_ylabel = lambda mode: 'Accuracy' if mode in ['acc', 'val_acc'] else 'Loss'
        plt.title(title)
        plt.ylabel(get_ylabel(mode))
        plt.xlabel('Epochs')
        
        colormap = cm.brg(np.linspace(0, 1, len(labels)))
        if complementary:
            plt.legend(handles=get_legend(colormap, labels), loc='upper right')
        else:
            plt.legend(handles=get_legend(colormap, labels, lines=False), loc='upper right')

        linestyles = ['--', '-']
        alphas = [1, 0.3]
        if mode.startswith('val_'): ind = 1
        else: ind = 0
        color_iter = iter(colormap)
        for val in histories:
            color = next(color_iter)
            plt.plot(val[mode], linestyle=linestyles[ind], color=color, alpha=alphas[ind])
        if complementary:
            if ind == 1:
                mode_2 = mode[4:]
            else:
                mode_2 = 'val_{}'.format(mode)
            color_iter = iter(colormap)
            for val in histories:
                color = next(color_iter)
                plt.plot(val[mode_2], '-', color=color, alpha=0.3)

def plot_acc_loss(histories, labels, title, prefix_1, prefix_2=None):
    """Plots both accuracy and validation traces for given histories.
    
    Params:
    -------
    histories - list of histories (keras.History.history or Extended History),
    labels - list of labels (for the legend),
    title - string, title of the plot,
    prefix_1 - {'', 'val_'}, the first prefix,
    prefix_2 - {'', 'val_'}, the second prefix, optional."""
    
    fig = plt.figure(figsize=(16,8))
    ax = []
    ax.append(fig.add_subplot(211))
    ax.append(fig.add_subplot(212))

    ax[0].set_title(title)
    ax[0].set_ylabel('Accuracy')
    ax[1].set_ylabel('Loss')
    ax[0].set_xlabel('Epochs')
    ax[1].set_xlabel('Epochs')

    colormap = cm.brg(np.linspace(0, 1, len(labels)))
    
    for i, mode in enumerate(['acc', 'loss']):
        color_iter = iter(colormap)
        linestyles = ['--', '-']
        alphas = [1, 0.3]
        if prefix_1 == 'val_':
            ind = 1
        else:
            ind = 0
        for val in histories:
            color = next(color_iter)
            ax[i].plot(val[prefix_1 + mode], linestyle=linestyles[ind], color=color, alpha=alphas[ind])
        del linestyles[ind]
        del alphas[ind]
        if prefix_2:
            color_iter = iter(colormap)
            for val in histories:
                color = next(color_iter)
                ax[i].plot(val[prefix_2 + mode], linestyles[0], color=color, alpha=alphas[0])

    # Put the legend to the right of the current axis
    for i in [0, 1]:
        box = ax[i].get_position()
        ax[i].set_position([box.x0, box.y0, box.width * 0.8, box.height])
        
    if prefix_2:
        legend = get_legend(colormap, labels)
    else:
        legend = get_legend(colormap, labels, lines=False)
    ax[0].legend(handles=legend, loc='upper right', bbox_to_anchor=(1.145, 1.0))

**NB:** Helper ↓ from: https://github.com/keras-team/keras/issues/341

In [ ]:
def shuffle_weights(model, weights=None):
    """Randomly permute the weights in `model`, or the given `weights`.

    This is a fast approximation of re-initializing the weights of a model.

    Assumes weights are distributed independently of the dimensions of the weight tensors
      (i.e., the weights have the same distribution along each dimension).

    :param Model model: Modify the weights of the given model.
    :param list(ndarray) weights: The model's weights will be replaced by a random permutation of these weights.
      If `None`, permute the model's current weights.
    """
    if weights is None:
        weights = model.get_weights()
    weights = [np.random.permutation(w.flat).reshape(w.shape) for w in weights]
    # Faster, but less random: only permutes along the first dimension
    # weights = [np.random.permutation(w) for w in weights]
    model.set_weights(weights)

In [ ]:
class ExtendedHistory:
    def __init__(self, history_=None, fname=None):
        if history_:
            self.history = history_
        elif fname:
            with open(fname, 'r') as file:
                self.history = json.load(file)
        else:
            self.history = {
                'loss': [],
                'val_loss': [],
                'acc': [],
                'val_acc': []
            }
        
    def add_hist(self, history):
        self.history['acc'] = list(np.concatenate((self.history['acc'], history.history['acc'])))
        self.history['val_acc'] = list(np.concatenate((self.history['val_acc'], history.history['val_acc'])))
        self.history['loss'] = list(np.concatenate((self.history['loss'], history.history['loss'])))
        self.history['val_loss'] = list(np.concatenate((self.history['val_loss'], history.history['val_loss'])))
        
    def save(self, fname):
        with open(fname, 'w') as file:
            file.write(json.dumps(self.history))
        
    def plot(self, title):
        fig, ax1, ax2 = prepare_standardplot(title, 'epoch')
        ax1.plot(self.history['loss'], label = "training")
        ax1.plot(self.history['val_loss'], label = "validation")
        ax2.plot(self.history['acc'], label = "training")
        ax2.plot(self.history['val_acc'], label = "validation")
        finalize_standardplot(fig, ax1, ax2)
#        return fig # Mute for %matplotinline

In [ ]:
class AppendedHistory:
    def __init__(self, varname=None, history_=None, fname=None):
        if history_:
            self.history = history_
        elif fname:
            with open(fname, 'r') as file:
                self.history = json.load(file)
        else:
            self.history = {
                'loss': [],
                'val_loss': [],
                'acc': [],
                'val_acc': [],
                'var': []
            }
        self.varname = varname
        
    def append_hist(self, var, history):
        """Append fitting history for a new variable."""
        for key in self.history.keys():
            if key != 'var':
                self.history[key].append(history.history[key])
            else:
                self.history['var'].append(var)
        
    def add_hist(self, var, history):
        """Add further fitting history to a certain variable."""
        ind = self.history['var'].index(var)
        for key in self.history.keys():
            if key != 'var':
                self.history[key][ind] = np.concatenate((self.history[key][ind], history.history[key])).tolist()
        
    def save(self, fname):
        with open(fname, 'w') as file:
            file.write(json.dumps(self.history))
            
    def get_single_var(self, var):
        new_hist = AppendedHistory()
        ind = self.history['var'].index(var)
        for key in self.history.keys():
            new_hist.history[key] = self.history[key][ind]
        return new_hist
    
    def get_slice(self, slice_):
        new_hist = AppendedHistory()
        for key in self.history.keys():
            new_hist.history[key] = self.history[key][slice_]
        return new_hist
        
    def plot(self, title, mode, complementary=True):
        get_ylabel = lambda mode: 'Accuracy' if mode in ['acc', 'val_acc'] else 'Loss'
        plt.title(title)
        plt.ylabel(get_ylabel(mode))
        plt.xlabel('Epochs')
        
        colormap = cm.gist_rainbow(np.linspace(0, 1, len(self.history['var'])))
        if complementary:
            plt.legend(handles=get_legend(colormap, self.history['var']), loc='upper right')
        else:
            plt.legend(handles=get_legend(colormap, self.history['var'], lines=False), loc='upper right')

        linestyles = ['--', '-']
        alphas = [1, 0.3]
        if mode.startswith('val_'): ind = 1
        else: ind = 0
        color_iter = iter(colormap)
        for val in self.history[mode]:
            color = next(color_iter)
            plt.plot(val, linestyle=linestyles[ind], color=color, alpha=alphas[ind])
        if complementary:
            if ind == 1:
                mode_2 = mode[4:]
            else:
                mode_2 = 'val_{}'.format(mode)
            color_iter = iter(colormap)
            for val in self.history[mode_2]:
                color = next(color_iter)
                plt.plot(val, '-', color=color, alpha=0.3)
            
    def plot_acc_loss(self, title, prefix_1, prefix_2=None):
        """Plots both accuracy and validation traces for given histories.
    
        Params:
        -------
        title - string, title of the plot,
        prefix_1 - {'', 'val_'}, the first prefix,
        prefix_2 - {'', 'val_'}, the second prefix, optional."""
        
        fig = plt.figure(figsize=(16,8))
        ax = []
        ax.append(fig.add_subplot(211))
        ax.append(fig.add_subplot(212))

        ax[0].set_title(title)
        ax[0].set_ylabel('Accuracy')
        ax[1].set_ylabel('Loss')
        ax[0].set_xlabel('Epochs')
        ax[1].set_xlabel('Epochs')

        colormap = cm.gist_rainbow(np.linspace(0, 1, len(self.history['var'])))
        
        for i, mode in enumerate(['acc', 'loss']):
            color_iter = iter(colormap)
            linestyles = ['--', '-']
            alphas = [1, 0.3]
            if prefix_1 == 'val_': ind = 1
            else: ind = 0
            for val in self.history[prefix_1 + mode]:
                color = next(color_iter)
                ax[i].plot(val, linestyle=linestyles[ind], color=color, alpha=alphas[ind])
            del linestyles[ind]
            del alphas[ind]
            if prefix_2:
                color_iter = iter(colormap)
                for val in self.history[prefix_2 + mode]:
                    color = next(color_iter)
                    ax[i].plot(val, linestyles[0], color=color, alpha=alphas[0])
           
        for i in [0, 1]:
            box = ax[i].get_position()
            ax[i].set_position([box.x0, box.y0, box.width * 0.8, box.height])

        # Put a legend to the right of the current axis
        if prefix_2:
            legend = get_legend(colormap, self.history['var'])
        else:
            legend = get_legend(colormap, self.history['var'], lines=False)
        ax[0].legend(handles=legend, loc='upper right', bbox_to_anchor=(1.135, 1.0))
        
    def get_ist(self, i):
        ist = {}
        for key in self.history.keys():
            if key != 'var':
                ist[key] = []
                for hist in self.history[key]:
                    ist[key].append(hist[i])
            else:
                continue
        return ist
    
    def get_best(self):
        best = {}
        for key in self.history.keys():
            if key == 'val_loss':
                best[key] = []
                for hist in self.history[key]:
                    best[key].append(min(hist))
                    best_ind = hist.index(min(hist))
            else:
                continue
        for key in self.history.keys():
            if (key != 'val_loss') and (key != 'var'):
                best[key] = []
                for hist in self.history[key]:
                    best[key].append(hist[best_ind])
        return best

### Loading and preprocessing the data

The data consists of RGB color images with 32x32 pixels, loaded into an array of dimension 32x32x3x(number of images). We convert them to grayscale (using [this method](https://en.wikipedia.org/wiki/SRGB#The_reverse_transformation)) and we downsample them to images of 16x16 pixels by averaging over patches of 2x2 pixels.

With these preprocessing steps we obviously remove some information that could be helpful in classifying the images. But, since the processed data is much lower dimensional, the fitting procedures converge faster. This is an advantage in situations like here (or generally when prototyping), where we want to try many different things without having to wait too long for computations to finish. After having gained some experience, one may want to go back to work on the 32x32 RGB images.


In [ ]:
# convert RGB images x to grayscale using the formula for Y_linear in https://en.wikipedia.org/wiki/Grayscale#Colorimetric_(perceptual_luminance-preserving)_conversion_to_grayscale
def grayscale(x):
    x = x.astype('float32')/255
    x = np.piecewise(x, [x <= 0.04045, x > 0.04045], 
                        [lambda x: x/12.92, lambda x: ((x + .055)/1.055)**2.4])
    return .2126 * x[:,:,0,:] + .7152 * x[:,:,1,:]  + .07152 * x[:,:,2,:]

def downsample(x):
    return sum([x[i::2,j::2,:] for i in range(2) for j in range(2)])/4

def preprocess(data):
    gray = grayscale(data['X'])
    downsampled = downsample(gray)
    return (downsampled.reshape(16*16, gray.shape[2]).transpose(),
            data['y'].flatten() - 1)


data_train = scipy.io.loadmat('housenumbers/train_32x32.mat')
data_test = scipy.io.loadmat('housenumbers/test_32x32.mat')

x_train_all, y_train_all = preprocess(data_train)
x_test_all, y_test_all = preprocess(data_test)

### Selecting a subset of classes

We furter reduce the size of the dataset (and thus reduce computation time) by selecting only the 5 (out of 10 digits) in `subset_of_classes`.

In [ ]:
def extract_classes(x, y, classes):
    indices = []
    labels = []
    count = 0
    for c in classes:
        tmp = np.where(y == c)[0]
        indices.extend(tmp)
        labels.extend(np.ones(len(tmp), dtype='uint8') * count)
        count += 1
    return x[indices], labels

x_train, y_train = extract_classes(x_train_all, y_train_all, subset_of_classes)
x_test, y_test = extract_classes(x_test_all, y_test_all, subset_of_classes)

Let us plot some examples now. The green digit at the bottom left of each image indicates the corresponding label in `y_test`.
For further usage of the function `plot_some_samples`, please have a look at its definition in the plotting section.

In [ ]:
plot_some_samples(x_test, y_test, label_mapping = subset_of_classes)

To prepare for fitting we transform the labels to one hot coding, i.e. for 5 classes, label 2 becomes the vector [0, 0, 1, 0, 0] (python uses 0-indexing).

In [ ]:
y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)

## Exercise 1: No hidden layer

### Ex 1: Description

Define and fit a model without a hidden layer. 

1. Use the `softmax` activation for the output layer.
2. Use the `categorical_crossentropy` loss.
3. Add the `accuracy` metric to the metrics.
4. Choose stochastic gradient descent (`SGD`) for the optimizer.
5. Choose a minibatch size of 128.
6. Fit for as many epochs as needed to see no further decrease in the validation loss.
7. Plot the output of the fitting procedure (a history object) using the function `plot_history` defined above.
8. Determine the indices of all test images that are misclassified by the fitted model and plot some of them using the function 
   `plot_some_samples(x_test, y_test, yhat_test, error_indices, label_mapping = subset_of_classes)`


Hints:
* Read the keras docs, in particular [Getting started with the Keras Sequential model](https://keras.io/getting-started/sequential-model-guide/).
* Have a look at the keras [examples](https://github.com/keras-team/keras/tree/master/examples), e.g. [mnist_mlp](https://github.com/keras-team/keras/blob/master/examples/mnist_mlp.py).

### Ex 1: Solution

In [ ]:
activation = 'softmax' #1. use softmax
loss = 'categorical_crossentropy' #2. use categorical_crossentropy for loss
metrics=['accuracy'] #3. add 'accuray' to metrics
optimizer = SGD() #4, SGD as optimizer
minibatch = 128 #5. define minibatch size

num_classes = y_test.shape[1]

model = Sequential()

model.add(Dense(num_classes, activation=activation, input_dim=x_train.shape[1]))

model.compile(loss=loss, 
              optimizer=optimizer,
              metrics=metrics)

epochs = 600 #6. fit for epochs when we don't see further decrease ca 500 epochs in.
history = model.fit(x_train, y_train, batch_size=minibatch, epochs=epochs, validation_data=(x_test, y_test), verbose=0) #validation split

#7. plot history
plot_history(history, "no hidden layer model")
plt.show()

#8. plot missclassified
pred = model.predict(x_test)
indexes = np.nonzero(pred.argmax(axis=1) != y_test.argmax(axis=1))[0]
plot_some_samples(x_test, y_test, pred, indexes, label_mapping = subset_of_classes)

## Exercise 2: One hidden layer, different optizimizers
### Ex 2: Description

Train a network with one hidden layer and compare different optimizers.

1. Use one hidden layer with 64 units and the `'relu'` activation. Use the [summary method](https://keras.io/models/about-keras-models/) to inspect your model.
2. Fit the model for 50 epochs with different learning rates of stochastic gradient descent and answer the question below.
3. Replace the stochastic gradient descent optimizer with the [Adam optimizer](https://keras.io/optimizers/#adam).
4. Plot the learning curves of SGD with a reasonable learning rate together with the learning curves of Adam in the same figure. Take care of a reasonable labeling of the curves in the plot.

### Ex 2: Solution

In [ ]:
one_hl = Sequential()
one_hl.add(Dense(64, input_dim=x_train.shape[1], activation='relu'))
one_hl.add(Dense(5, input_dim=x_train.shape[1], activation='softmax'))
one_hl.summary()

In [ ]:
lrs = [0.00001, 0.00005, 0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1.0, 5.0, 0.7, 0.9, 0.6]
inds = np.argsort(lrs)
inds

In [ ]:
apphist_onehl = AppendedHistory()
weights = one_hl.get_weights()

for r in sorted(lrs):
    sgd = SGD(lr=r)
    one_hl.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['acc'])
    one_hl.set_weights(weights)
    %time hist_onehl = one_hl.fit(x_train, y_train, epochs=50, batch_size=128, \
                                   verbose=0, validation_data=(x_test, y_test))
    apphist_onehl.append_hist(r, hist_one_hl)

In [ ]:
np.save('output_Olesya/apphist_onehl_sameinitws_weights', weights)
apphist_onehl.save('output_Olesya/apphist_onehl_sameinitws.txt')

In [ ]:
apphist_onehl = AppendedHistory(varname='sgd lr', fname='output_Olesya/apphist_onehl_sameinitws.txt')

In [ ]:
# HERE: Plotting yet to be corrected !!!

In [ ]:
one_hl.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
one_hl.set_weights(weights)
%time hist_one_hl_adam = one_hl.fit(x_train, y_train, epochs=50, batch_size=128, \
                                    verbose=0, validation_data=(x_test, y_test))

In [ ]:
histories = [hist_one_hl_adam.history, apphist_onehl.get_single_var(0.5).history]
labels = ['Adam', 'SGD, lr=0.5']

In [ ]:
plot_acc_loss(histories, labels, 'Adam vs Best SGD', '', 'val_')

**Question**: What happens if the learning rate of SGD is A) very large B) very small? Please answer A) and B) with one full sentence (double click this markdown cell to edit).

**Answer**:

A) The model **does not converge**, i.e. it 'sits' at a very low accuracy and a very high loss.

B) The model converges **very slowly and non-optimally** (to a non-optimal accuracy and loss).

## Exercise 3: Overfitting and early stopping with Adam

### Ex 3: Description

Run the above simulation with Adam for sufficiently many epochs (be patient!) until you see clear overfitting.

1. Plot the learning curves of a fit with Adam and sufficiently many epochs and answer the questions below.

A simple, but effective mean to avoid overfitting is early stopping, i.e. a fit is not run until convergence but stopped as soon as the validation error starts to increase. We will use early stopping in all subsequent exercises.

### Ex 3: Solution

In [ ]:
exthist_longadam = ExtendedHistory()

In [ ]:
shuffle_weights(one_hl)
np.save('output_Olesya/exthist_longadam_weights', one_hl.get_weights())

In [ ]:
#stop = keras.callbacks.EarlyStopping(monitor='val_acc', min_delta=0.001, patience=10, verbose=0, mode='auto')

#one_hl.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
%time hist_one_hl_long_adam = one_hl.fit(x_train, y_train, epochs=400, batch_size=128, \
                                         verbose=0, validation_data=(x_test, y_test))
exthist_longadam.add_hist(hist_one_hl_long_adam)

In [ ]:
exthist_longadam.save('output_Olesya/exthist_longadam.txt')

In [ ]:
exthist_longadam = ExtendedHistory(fname='output_Olesya/exthist_longadam.txt')

In [ ]:
exthist_longadam.plot('Overfitting with Adam')

**Question 1**: At which epoch (approximately) does the model start to overfit? Please answer with one full sentence.

**Answer**: Around epoch 80.

**Question 2**: Explain the qualitative difference between the loss curves and the accuracy curves with respect to signs of overfitting. Please answer with at most 3 full sentences.

**Answer**:
- At the moment when overfitting starts, the **validation loss** is at its minimum while the **training loss** continues to decrease exponentially (as $exp(-x)$).
- By the time the model starts to overfit, the **validation accuracy** is in the very close neighbourhood of its asymptote behaving as $1 - exp(-x)$ while the **training accuracy** continues to increase logarithmically (=very slowly).

## Exercise 4: Model performance as a function of number of hidden neurons

### Ex 4: Description

Investigate how the best validation loss and accuracy depends on the number of hidden neurons in a single layer.

1. Fit a reasonable number of models with different hidden layer size (between 10 and 1000 hidden neurons) for a fixed number of epochs well beyond the point of overfitting.
2. Collect some statistics by fitting the same models as in 1. for multiple initial conditions. Hints: 1. If you don't reset the random seed, you get different initial conditions each time you create a new model. 2. Let your computer work while you are asleep.
3. Plot summary statistics of the final validation loss and accuracy versus the number of hidden neurons. Hint: [boxplots](https://matplotlib.org/examples/pylab_examples/boxplot_demo.html) (also [here](https://matplotlib.org/api/_as_gen/matplotlib.axes.Axes.boxplot.html?highlight=boxplot#matplotlib.axes.Axes.boxplot)) are useful. You may also want to use the matplotlib method set_xticklabels.
4. Plot summary statistics of the loss and accuracy for early stopping versus the number of hidden neurons.

### Ex 4: Solution

**Training**

In [ ]:
for i in range(6, 11):
    apphist_densities = AppendedHistory(varname='weights')
    for j in range(10):
        start_time = timeit.default_timer()
        one_hl = Sequential()
        one_hl.add(Dense(2**i, input_dim=x_train.shape[1], activation='relu'))
        one_hl.add(Dense(5, input_dim=x_train.shape[1], activation='softmax'))
        np.save('output_Olesya/weights_{}-{}'.format(2**i, j), one_hl.get_weights())
        one_hl.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
        hist_one_hl_density = one_hl.fit(x_train, y_train, epochs=300, batch_size=128, \
                                        verbose=0, validation_data=(x_test, y_test))
        apphist_densities.append_hist(j, hist_one_hl_density)
        elapsed = timeit.default_timer() - start_time
        print('{}: {}'.format(j, elapsed))
    apphist_densities.save('output_Olesya/apphist_densities_{}_1.txt'.format(2**i))

**Plotting**

**NB**: For proper comparison, we plot statistics not only for validation, but also for training.

In [ ]:
fpath = 'output_Olesya/densities/'
box_dict = {
    'acc': [AppendedHistory(fname='{}apphist_densities_{}.txt'.format(fpath, 2**i)).get_ist(-1)['acc'] \
             for i in range(3, 11)],
    'val_acc': [AppendedHistory(fname='{}apphist_densities_{}.txt'.format(fpath, 2**i)).get_ist(-1)['val_acc'] \
             for i in range(3, 11)],
    'loss': [AppendedHistory(fname='{}apphist_densities_{}.txt'.format(fpath, 2**i)).get_ist(-1)['loss'] \
             for i in range(3, 11)],
    'val_loss': [AppendedHistory(fname='{}apphist_densities_{}.txt'.format(fpath, 2**i)).get_ist(-1)['val_loss'] \
             for i in range(3, 11)]
}

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(20, 8))

bplots = []
for key in box_dict.keys():
    if key.endswith('loss'):
        bplots.append(ax[0].boxplot(box_dict[key], patch_artist=True))
    else:
        bplots.append(ax[1].boxplot(box_dict[key], patch_artist=True))
        
ax[0].set_xlabel('HL Size, $N=2^{x+2}$')
ax[1].set_xlabel('HL Size, $N=2^{x+2}$')
ax[0].set_ylabel('Loss')
ax[1].set_ylabel('Accuracy')
ax[0].set_title('Final Loss Stats: 8 models with 1 HL of size N, 10 iterations each')
ax[1].set_title('Final Accuracy Stats: 8 models with 1 HL of size N, 10 iterations each')
#ax[0].set_yscale('log')

colormap = cm.Wistia(np.linspace(0, 1, len(box_dict)))

ax[0].legend(handles=get_legend(colormap[2:], ['acc', 'val_acc'], lines=False), loc='upper left')
ax[1].legend(handles=get_legend(colormap[:2], ['loss', 'val_loss'], lines=False), loc='upper left')

for ind, bplot in enumerate(bplots):
    for patch in bplot['boxes']:
        patch.set_facecolor(colormap[ind])

In [ ]:
fpath = 'output_Olesya/densities/'
box_dict = {
    'acc': [AppendedHistory(fname='{}apphist_densities_{}.txt'.format(fpath, 2**i)).get_best()['acc'] \
             for i in range(3, 11)],
    'val_acc': [AppendedHistory(fname='{}apphist_densities_{}.txt'.format(fpath, 2**i)).get_best()['val_acc'] \
             for i in range(3, 11)],
    'loss': [AppendedHistory(fname='{}apphist_densities_{}.txt'.format(fpath, 2**i)).get_best()['loss'] \
             for i in range(3, 11)],
    'val_loss': [AppendedHistory(fname='{}apphist_densities_{}.txt'.format(fpath, 2**i)).get_best()['val_loss'] \
             for i in range(3, 11)]
}

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(20, 8))

bplots = []
for key in box_dict.keys():
    if key.endswith('loss'):
        bplots.append(ax[0].boxplot(box_dict[key], patch_artist=True))
    else:
        bplots.append(ax[1].boxplot(box_dict[key], patch_artist=True))
        
ax[0].set_xlabel('HL Size, $N=2^{x+2}$')
ax[1].set_xlabel('HL Size, $N=2^{x+2}$')
ax[0].set_ylabel('Loss')
ax[1].set_ylabel('Accuracy')
ax[0].set_title('Best Loss Stats: 8 models with 1 HL of size N, 10 iterations each')
ax[1].set_title('Best Accuracy Stats: 8 models with 1 HL of size N, 10 iterations each')
#ax[0].set_yscale('log')

colormap = cm.Wistia(np.linspace(0, 1, len(box_dict)))

ax[0].legend(handles=get_legend(colormap[2:], ['acc', 'val_acc'], lines=False), loc='upper left')
ax[1].legend(handles=get_legend(colormap[:2], ['loss', 'val_loss'], lines=False), loc='upper left')

for ind, bplot in enumerate(bplots):
    for patch in bplot['boxes']:
        patch.set_facecolor(colormap[ind])

## Exercise 5: Comparison to deep models

### Ex 5: Description

Instead of choosing one hidden layer (with many neurons) you experiment here with multiple hidden layers (each with not so many neurons).

1. Fit models with 2, 3 and 4 hidden layers with approximately the same number of parameters as a network with one hidden layer of 100 neurons. Hint: Calculate the number of parameters in a network with input dimensionality N_in, K hidden layers with N_h units, one output layer with N_out dimensions and solve for N_h. Confirm you result with the keras method model.summary().
2. Run each model multiple times with different initial conditions and plot summary statistics of the best validation loss and accuracy versus the number of hidden layers.

### Ex 5: Solution


a network with one hidden layer of 100 neurons has

$$100*(|input|+1) + (100+1)*|output| = 100 * (256+1) + (100+1)*5 = 26,205$$

parameters.

Hence for a system with 2 hidden layers to have a similar number of parameters we should have that:

$$ x*(|input|+1)+(x+1)*y + (y+1)*|output| = 26,205$$

For 3 hidden layers we have:

$$ x*(|input|+1)+(x+1)*y +(y+1)*z+(z+1)*|output| = 26,205$$

For 4 we have:

$$ x*(|input|+1)+(x+1)*y + (y+1)*z + (z+1) * \alpha + (\alpha +1)*|output| = 26,205$$ 


We now simplify these by setting the layersize for each to N_h, using the notation suggested above:

$$ (N_{in}+1)*N_h +(N_h+1)*N_h*(K-1) + N_{out}*(N_h+1) = 26,205 $$

We solve for $N_h$:
$$N_h = \frac{(N_{in}+N_{out}+K)\pm \sqrt{(N_{in}+N_{out}+K)^2-4(K-1)*(N_{out}-26205)}}{2(K-1)}
$$

(obviously the only valid solution is N_h > 0)

- Hence for 2 layers:  ca 77
- For 3 layers: ca 66
- For 4 layer: ca 59

**Training**

**NB:** More iterations yet to come.

In [ ]:
dens_list = [[78, 73], [66, 66, 67], [59, 59, 59, 59]]

for denss in dens_list:
    apphist_layers = AppendedHistory()
    for j in range(10):
        start_time = timeit.default_timer()
        equal_n_params = Sequential()
        equal_n_params.add(Dense(denss[0], input_dim=x_train.shape[1], activation='relu'))
        if len(denss) > 1:
            for dens in denss[1:]:
                equal_n_params.add(Dense(dens, activation='relu'))
        equal_n_params.add(Dense(5, input_dim=x_train.shape[1], activation='softmax'))
        equal_n_params.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
        hist_equal_n_params = equal_n_params.fit(x_train, y_train, epochs=300, batch_size=128, \
                                        verbose=0, validation_data=(x_test, y_test))
        apphist_layers.append_hist(j, hist_equal_n_params)
        elapsed = timeit.default_timer() - start_time
        print('{}: {}'.format(j, elapsed))
    apphist_layers.save('/output/apphist_layers_{}.txt'.format(denss[0]))

**Plotting**

In [ ]:
fpath = 'output_Olesya/layers/'
box_dict = {
    'acc': [AppendedHistory(fname='{}apphist_layers_{}.txt'.format(fpath, i)).get_best()['acc'] \
             for i in range(2, 5)],
    'val_acc': [AppendedHistory(fname='{}apphist_layers_{}.txt'.format(fpath, i)).get_best()['val_acc'] \
             for i in range(2, 5)],
    'loss': [AppendedHistory(fname='{}apphist_layers_{}.txt'.format(fpath, i)).get_best()['loss'] \
             for i in range(2, 5)],
    'val_loss': [AppendedHistory(fname='{}apphist_layers_{}.txt'.format(fpath, i)).get_best()['val_loss'] \
             for i in range(2, 5)]
}

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(20, 8))

bplots = []
for key in box_dict.keys():
    if key.endswith('loss'):
        bplots.append(ax[0].boxplot(box_dict[key], patch_artist=True))
    else:
        bplots.append(ax[1].boxplot(box_dict[key], patch_artist=True))
        
ax[0].set_xlabel('# of layers, x+1')
ax[1].set_xlabel('# of layers, x+1')
ax[0].set_ylabel('Loss')
ax[1].set_ylabel('Accuracy')
ax[0].set_title('Best Loss Stats: 2-3-4 layers, 10 iterations each')
ax[1].set_title('Best Accuracy Stats: 2-3-4 layers, 10 iterations each')
#ax[0].set_yscale('log')

colormap = cm.Wistia(np.linspace(0, 1, len(box_dict)))

ax[0].legend(handles=get_legend(colormap[2:], ['acc', 'val_acc'], lines=False), loc='upper left')
ax[1].legend(handles=get_legend(colormap[:2], ['loss', 'val_loss'], lines=False), loc='upper left')

for ind, bplot in enumerate(bplots):
    for patch in bplot['boxes']:
        patch.set_facecolor(colormap[ind])

## Exercise 6: Tricks (regularization, batch normalization, dropout)

### Ex 6: Description

Overfitting can also be counteracted with regularization and dropout. Batch normalization is supposed to mainly decrease convergence time.

1. Try to improve the best validation scores of the model with 1 layer and 100 hidden neurons and the model with 4 hidden layers. Experiment with batch_normalization layers, dropout layers and l1- and l2-regularization on weights (kernels) and biases.
2. After you have found good settings, plot for both models the learning curves of the naive model you fitted in the previous exercises together with the learning curves of the current version.
3. For proper comparison, plot also the learning curves of the two current models in a third figure.

### Ex 6: Solution

Trying l1 and l2 regularisation we find that l1 regularisation does not work well for this data. l2 works, but drop out layers are best preforming. 

In [ ]:
# model with 1 hidden layer
# add dropout to each layer, use some batch normalisation for speed-up
epochs = 150

model = Sequential()
model.add(Dense(100, activation=activation, input_dim=x_train.shape[1]))
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(Dense(num_classes, activation='softmax', name='outputlayer'))

model.compile(loss=loss, 
              optimizer=optimizer,
              metrics=metrics)

model.summary()
history = model.fit(x_train, y_train, 
                      batch_size=minibatch, 
                      epochs=epochs, 
                      validation_data=(x_test, y_test), 
                      verbose=0)

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
# model with 4 hidden layers
# add dropout to each layer, use some batch normalisation for speed-up
model_4 = Sequential()
model_4.add(Dense(59, activation=activation, input_dim=x_train.shape[1]))
model_4.add(BatchNormalization())
model_4.add(Dropout(0.125))
model_4.add(Dense(59, activation=activation, input_dim=59))
model_4.add(Dropout(0.125))
model_4.add(Dense(59, activation=activation, input_dim=59))
model_4.add(Dropout(0.125))
model_4.add(Dense(59, activation=activation, input_dim=59))
model_4.add(Dropout(0.25))
model_4.add(BatchNormalization())
model_4.add(Dense(num_classes, activation='softmax', name='outputlayer'))

model_4.compile(loss=loss, 
              optimizer=optimizer,
              metrics=metrics)
model_4.summary()
history_4 = model_4.fit(x_train, y_train, 
                      batch_size=minibatch, 
                      epochs=epochs, 
                      validation_data=(x_test, y_test), 
                      verbose=0)

#get score
score = model_4.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
# 2. plot learning curves vs so far best preforming model


In [ ]:
#3. learning curves of two current models in a third figures
plot_history(history_4, "4 hidden layers with dropout")
plt.show()

plot_history(history, "1 hidden layer, 100 neurons with dropout")
plt.show()

## Exercise 7: Convolutional networks

### Ex 7: Description

Convolutional neural networks have an inductive bias that is well adapted to image classification.

1. Design a convolutional neural network, play with the parameters and fit it. Hint: You may get valuable inspiration from the keras [examples](https://github.com/keras-team/keras/tree/master/examples), e.g. [mnist_cnn](https://github.com/keras-team/keras/blob/master/examples/mnist_cnn.py).
2. Plot the learning curves of the convolutional neural network together with the so far best performing model.

### Ex 7: Solution

In [ ]:
#  1. Neural network design.
batch_size = 128
num_classes = 5
epochs = 12

img_rows, img_cols = 16, 16

#change input shape
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

model = Sequential()

#four convolutional layers + regularisation methods
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape, padding='same'))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))

model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.125))

model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu'))

model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.125))
model.add(Flatten())

#neural network on top
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss="categorical_crossentropy",
              optimizer=Adam(),
              metrics=['accuracy'])


model.summary()

#fit model
history_cnn = model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))


#plot history
plot_history(history_cnn, 'Convol neural network')
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1]) #ca 0.95

In [ ]:
#show missclassified, we see missclassified are bad samples
pred = model.predict(x_test)
indexes = np.nonzero(pred.argmax(axis=1) != y_test.argmax(axis=1))[0]
plot_some_samples(x_test, y_test, pred, indexes, label_mapping = subset_of_classes)

In [ ]:
# 2. plot learning curves vs so far best preforming model